Random State for reproduceability

In [1]:
dagstuhl_seed=23372

# Data

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('../../data/preprocessed.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df_train = pd.read_csv('../../data/preprocessed-train.csv')
df_train = df_train.loc[:, ~df_train.columns.str.contains('^Unnamed')]

df_test = pd.read_csv('../../data/preprocessed-test.csv')
df_test = df_test.loc[:, ~df_test.columns.str.contains('^Unnamed')]

df.tail()

,label,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
4306,inactive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4307,inactive,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4308,inactive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4309,inactive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4310,inactive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
y = df_train['label']
X = df_train.loc[:, df_train.columns != "label"]
X.to_numpy().shape

(3592, 1024)

# AutoML Libraries

In [5]:
time_min = 60
time_sec = time_min * 60

## H2O

In [6]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads=-1, log_level="TRACE")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.8.1" 2023-08-24; OpenJDK Runtime Environment (build 17.0.8.1+1-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 17.0.8.1+1-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpl0j89hyt
  JVM stdout: /tmp/tmpl0j89hyt/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpl0j89hyt/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,30 days
H2O_cluster_name:,H2O_from_python_unknownUser_kl88sr
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.887 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [7]:
hf = h2o.H2OFrame(df_train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
%%time
aml = H2OAutoML(max_runtime_secs=time_sec, seed=dagstuhl_seed, verbosity="debug")
aml.train(y = 'label' , training_frame = hf)

AutoML progress: |
07:03:53.262: Project: AutoML_2_20230922_70353
07:03:53.262: 5-fold cross-validation will be used.
07:03:53.262: Setting stopping tolerance adaptively based on the training frame: 0.016685216106649997
07:03:53.262: Build control seed: 23372
07:03:53.263: training frame: Frame key: AutoML_2_20230922_70353_training_Key_Frame__upload_bb830e471e8c01632129daebf3ef48ed.hex    cols: 1025    rows: 3592  chunks: 4    size: 403868  checksum: -87072914012808
07:03:53.263: validation frame: NULL
07:03:53.263: leaderboard frame: NULL
07:03:53.263: blending frame: NULL
07:03:53.263: response column: label
07:03:53.263: fold column: null
07:03:53.263: weights column: null
07:03:53.264: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_search (6g, 30w)]}, {GLM : [def_1 (1g, 10w)]}, {DRF : [def_1 (2g, 10w), XRT (3g, 10w)]}, {GBM : [def_5 (1g, 10w), def_2 (2g, 10w), def_3 (2g, 10w), def_4 (2g, 10w), def_1 (3g, 10w), grid_1 (4

key,value
Stacking strategy,cross_validation
Number of base models (used / total),4/6
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# DRF base models (used / total),1/2
# GLM base models (used / total),1/1
# DeepLearning base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [12]:
hf_test = h2o.H2OFrame(df_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [27]:
from sklearn.metrics import accuracy_score
y_pred = aml.predict(hf_test)
accuracy_score(df_test["label"], y_pred.as_data_frame()['predict']) 

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


0.9443671766342142

In [28]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_BestOfFamily_4_AutoML_2_20230922_70353,0.811383,0.206664,0.976683,0.445431,0.235991,0.0556919,503,0.04129,StackedEnsemble
StackedEnsemble_BestOfFamily_5_AutoML_2_20230922_70353,0.810375,0.207058,0.977054,0.449427,0.236346,0.0558594,394,0.04812,StackedEnsemble
StackedEnsemble_BestOfFamily_8_AutoML_2_20230922_70353,0.807147,0.209462,0.97838,0.45652,0.238308,0.0567908,443,0.050921,StackedEnsemble
StackedEnsemble_Best1000_1_AutoML_2_20230922_70353,0.806971,0.208146,0.978514,0.441735,0.237038,0.056187,3022,0.220452,StackedEnsemble
StackedEnsemble_AllModels_6_AutoML_2_20230922_70353,0.806611,0.209065,0.978407,0.443808,0.237722,0.0565117,20205,0.225715,StackedEnsemble
StackedEnsemble_BestOfFamily_7_AutoML_2_20230922_70353,0.806409,0.208378,0.976002,0.456369,0.236861,0.056103,4082,0.042157,StackedEnsemble
StackedEnsemble_AllModels_3_AutoML_2_20230922_70353,0.805646,0.208237,0.977067,0.447204,0.236675,0.0560149,2486,0.106873,StackedEnsemble
StackedEnsemble_AllModels_4_AutoML_2_20230922_70353,0.805497,0.208456,0.978005,0.441435,0.236875,0.0561095,2282,0.16563,StackedEnsemble
StackedEnsemble_BestOfFamily_2_AutoML_2_20230922_70353,0.804071,0.20998,0.977521,0.447354,0.238091,0.0566873,488,0.030631,StackedEnsemble
StackedEnsemble_AllModels_1_AutoML_2_20230922_70353,0.803543,0.210387,0.977933,0.447354,0.238497,0.0568807,566,0.043384,StackedEnsemble


In [29]:
m = aml.leader
m.params.keys()

dict_keys(['model_id', 'training_frame', 'response_column', 'validation_frame', 'blending_frame', 'base_models', 'metalearner_algorithm', 'metalearner_nfolds', 'metalearner_fold_assignment', 'metalearner_fold_column', 'metalearner_params', 'metalearner_transform', 'max_runtime_secs', 'weights_column', 'offset_column', 'seed', 'score_training_samples', 'keep_levelone_frame', 'export_checkpoints_dir', 'auc_type'])

In [31]:
# Get AutoML event log
log = aml.event_log
log

timestamp,level,stage,message,name,value
07:03:53.262,INFO,Workflow,Project: AutoML_2_20230922_70353,,
07:03:53.262,INFO,Validation,5-fold cross-validation will be used.,,
07:03:53.262,INFO,Validation,Setting stopping tolerance adaptively based on the training frame: 0.016685216106649997,,
07:03:53.262,INFO,Validation,Build control seed: 23372,,
07:03:53.263,INFO,DataImport,training frame: Frame key: AutoML_2_20230922_70353_training_Key_Frame__upload_bb830e471e8c01632129daebf3ef48ed.hex cols: 1025 rows: 3592 chunks: 4 size: 403868 checksum: -87072914012808,,
07:03:53.263,INFO,DataImport,validation frame: NULL,,
07:03:53.263,INFO,DataImport,leaderboard frame: NULL,,
07:03:53.263,INFO,DataImport,blending frame: NULL,,
07:03:53.263,INFO,DataImport,response column: label,,
07:03:53.263,INFO,DataImport,fold column: null,,


In [33]:
# Get training timing info
info = aml.training_info
info

{'creation_epoch': '1695366233',
 'start_epoch': '1695366233',
 'start_XGBoost_def_2': '1695366233',
 'start_GLM_def_1': '1695366243',
 'start_GBM_def_5': '1695366248',
 'start_StackedEnsemble_best_of_family_1': '1695366269',
 'start_XGBoost_def_1': '1695366270',
 'start_DRF_def_1': '1695366274',
 'start_GBM_def_2': '1695366287',
 'start_GBM_def_3': '1695366305',
 'start_GBM_def_4': '1695366328',
 'start_StackedEnsemble_best_of_family_2': '1695366349',
 'start_StackedEnsemble_all_2': '1695366350',
 'start_XGBoost_def_3': '1695366350',
 'start_DRF_XRT': '1695366357',
 'start_GBM_def_1': '1695366365',
 'start_DeepLearning_def_1': '1695366378',
 'start_StackedEnsemble_best_of_family_3': '1695366396',
 'start_StackedEnsemble_all_3': '1695366396',
 'start_XGBoost_grid_1': '1695366397',
 'start_GBM_grid_1': '1695366947',
 'start_DeepLearning_grid_1': '1695367281',
 'start_StackedEnsemble_best_of_family_4': '1695369014',
 'start_StackedEnsemble_all_4': '1695369015',
 'start_DeepLearning_grid_